In [57]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from random import randint

## Generate rotated card

In [58]:
img = cv2.imread("svgtest.png")

def rotate_image(image, rotation_angle, center_points=None):
    h, w = image.shape[:2]
    if center_points is None:
        cx, cy = (w // 2, h // 2)
    else:
        cx, cy = center_points
        
    # get rotation matrix (explained in section below)
    M = cv2.getRotationMatrix2D((cx, cy), rotation_angle, 1.0)

    # get cos and sin value from the rotation matrix
    cos, sin = abs(M[0, 0]), abs(M[0, 1])

    # calculate new width and height after rotation (explained in section below)
    newW = int((h * sin) + (w * cos))
    newH = int((h * cos) + (w * sin))

    # calculate new rotation center
    M[0, 2] += (newW / 2) - cx
    M[1, 2] += (newH / 2) - cy

    # use modified rotation center and rotation matrix in the warpAffine method
    return cv2.warpAffine(image, M, (newW, newH), borderMode=cv2.BORDER_CONSTANT,
                            borderValue=(255,255,255)) 
                

# new_image = rotate_image(img, 27)
# plt.imshow(new_image);

In [59]:
# gray_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)

def get_contours(new_image):
    mask = cv2.inRange(new_image, 0, 150)

    # apply morphology
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    clean = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15,15))
    clean = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # get external contours
    contours = cv2.findContours(clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]

    squares = []
    for c in contours:
        # get rotated rectangle from contour
        rot_rect = cv2.minAreaRect(c)
        if abs(rot_rect[1][0] - rot_rect[1][1]) < min(rot_rect[1]) / 50:
            squares.append(rot_rect)
    return squares

# result1 = np.full_like(gray_image, fill_value=255)
# squares = get_contours(gray_image)
# for rect in squares:
#     box = cv2.boxPoints(rect) # cv2.boxPoints(rect) for OpenCV 3.x
#     box = np.int0(box)
#     cv2.drawContours(result1, [box], 0, 0, -1)

# # plt.imshow(result1, cmap='gray', vmin=0, vmax=255)
# print(len(squares))

In [60]:
def calculate_delta(first_pos, second_pos):
        return abs(first_pos[0] - second_pos[0]), abs(first_pos[1] - second_pos[1])
        
def set_numeric_values_squares(contours):
    def is_similar(first_delta, second_delta):
        return abs(first_delta[0] - second_delta[0]) <= min(first_delta[0], second_delta[0]) / 100 and \
               abs(first_delta[1] - second_delta[1]) <= min(first_delta[1], second_delta[1]) / 100

    # Top       left: 4,                
    # Middle    left: 3
    # Bottom    left: 0,    middle 1,   right: 2
    # Page width = distance between 0 and 2
    # Page height = distance between 0 and 4
    
    # Find three squares lines 
    # (should be two, first vertical on left side, second horizontal on bottom)
    points = []
    for first_idx in range(len(contours)):
        first_point = contours[first_idx]
        for second_idx in range(first_idx + 1, len(contours)):
            second_point = contours[second_idx]
            for third_idx in range(second_idx + 1, len(contours)):
                third_point = contours[third_idx]
                fs_delta = calculate_delta(first_point[0], second_point[0])
                ft_delta = calculate_delta(first_point[0], third_point[0])
                st_delta = calculate_delta(second_point[0], third_point[0])
                if is_similar(fs_delta, st_delta):
                    points.append([first_point, second_point, third_point])
                elif is_similar(fs_delta, ft_delta):
                    points.append([second_point, first_point, third_point])  
                elif is_similar(st_delta, ft_delta):
                    points.append([first_point, third_point, second_point])
    
    assert len(points) == 2, "Didn't detect two lines of squares, can't rotate page"


    # Reorder points based on commonly shared sqare (0 index square)
    first_line = points[0]
    second_line = points[1]

    if first_line[0] == second_line[0]:
        pass
    elif first_line[2] == second_line[2]:
        first_line = first_line[::-1] 
        second_line = second_line[::-1] 
    elif first_line[0] == second_line[2]:
        second_line = second_line[::-1]
    elif first_line[2] == second_line[0]:
        first_line = first_line[::-1]
    
    # Calculate length of lines based on length of lines
    fl_x, fl_y = calculate_delta(first_line[0][0], first_line[2][0])
    sl_x, sl_y = calculate_delta(second_line[0][0], second_line[2][0])

    fl_dist = (fl_x ** 2 + fl_y ** 2) ** (1/2)
    sl_dist = (sl_x ** 2 + sl_x ** 2) ** (1/2)

    # Assign lines to proper values
    if fl_dist < sl_dist:
        horizontal_line = first_line
        vertical_line = second_line
    else:
        vertical_line = second_line
        horizontal_line = first_line

    # Assign indexes to proper squares
    numered_points = dict(enumerate([*horizontal_line, *vertical_line[1:]]))
    return numered_points

In [61]:
def find_middle_of_page(numered_points):
    zero_pos = np.array(numered_points[0][0])
    first_pos = np.array(numered_points[1][0])
    third_pos = np.array(numered_points[3][0])
    zero_to_one_distance = first_pos - zero_pos
    zero_to_three_distance = third_pos - zero_pos
    middle_from_one = first_pos + zero_to_three_distance
    middle_from_three = third_pos + zero_to_one_distance
    return (middle_from_one + middle_from_three) / 2
    
def find_angle_of_page(numered_squares):
    zero_square = np.array(numered_squares[0][0])
    first_square = np.array(numered_squares[1][0])
    opposite_to_angle = zero_square[0] - first_square[0]
    hypotenuse = sum((zero_square - first_square) ** 2) ** (1/2)
    return math.asin(opposite_to_angle/hypotenuse) * 180 / math.pi

# TODO Manage how to calculate angle
def calculate_quarter(numered_squares):
    zs_x, zs_y = numered_squares[0][0]
    fs_x, fs_y = numered_squares[1][0]
    if zs_x >= fs_x:
        if zs_y >= fs_y:
            return 2
        else:
            return 3
    else:
        if zs_y >= fs_y:
            return 1
        else:
            return 4

def modify_angle(new_angle, quarter):
    if new_angle == 0:
        return new_angle
    new_angle = int(new_angle + 0.5 * (new_angle / abs(new_angle)))
    if new_angle > 0:
        if new_angle < 45:
            new_angle += 90
        else:
            new_angle *= -1
    elif new_angle < 0:
        new_angle += 90
        if new_angle > 0:
            new_angle *= -1
    return new_angle   

def pipeline(parsed_image):
    if len(parsed_image.shape) == 3:
        parsed_image = cv2.cvtColor(parsed_image, cv2.COLOR_RGB2GRAY)
    squares = get_contours(parsed_image)
    numered_squares = set_numeric_values_squares(squares)
    quarter = calculate_quarter(numered_squares)
    middle_pos = find_middle_of_page(numered_squares)
    new_angle = find_angle_of_page(numered_squares)
    # new_angle = modify_angle(new_angle, 1)
    return new_angle, middle_pos, quarter

In [62]:
def validation(image):
    guesses_hits = []
    quarter_modifiers = [list() for _ in range(4)]
    shots = 100
    hits = 0
    test_angles = [0, 30, 45, 60, 90, 120, 135, 150, 180, 210, 225, 240, 270, 300, 315, 345]
    for random_angle in test_angles:
        # random_angle = randint(-90, 90)
        temp_image = rotate_image(image, random_angle)
        # plt.imshow(temp_image);
        detected_angle, _, quarter = pipeline(temp_image)
        # print(random_angle, detected_angle)
        if random_angle == -detected_angle:
            hits += 1
        else:
            quarter_modifiers[quarter - 1].append([-random_angle, detected_angle, -random_angle-detected_angle])
            guesses_hits.append([-random_angle, detected_angle, quarter])
    # print(guesses_hits)
    for quarter in quarter_modifiers:
        print(quarter)
    return float(hits/shots)

In [63]:
results = validation(img)
print(results)

[[0, -90.0, 90.0], [-30, -60.00004924919652, 30.000049249196522], [-45, -44.999988827296576, -1.1172703423767416e-05]]
[[-60, 60.0001101382345, -120.0001101382345], [-90, 90.0, -180.0]]
[[-120, 59.99987343367697, -179.99987343367695], [-135, 45.00000714413977, -180.00000714413977], [-150, 30.000131616248037, -180.00013161624804], [-180, 0.0, -180.0], [-240, 29.999664575968605, -269.9996645759686], [-270, 0.0, -270.0]]
[[-210, -29.99988359470522, -180.0001164052948], [-225, -45.000000893017486, -179.9999991069825], [-300, -29.99995523163402, -270.00004476836597], [-315, -45.000005586352266, -269.99999441364776], [-345, -74.99961168450425, -270.00038831549574]]
0.0


In [64]:
# new_angle, center_points = pipeline(new_image)
# result = rotate_image(new_image, angle)

# plt.imshow(result);